In [1]:
import pandas as pd

df = pd.read_csv("tmp/interventions.csv")
df.head()

,intervenant,parti,intervention
0,Yaël Braun-Pivet,REN,\r\nL'ordre du jour appelle la discussion du p...
1,Danièle Obono,LFI,"\r\nNon, pas question !"
2,Louis Boyard,LFI,"\r\nNon, ça ne va pas se passer comme ça ! Vou..."
3,Yaël Braun-Pivet,REN,"\r\nLa parole est à M. le ministre du travail,..."
4,André Chassaigne,GDR,\r\nSuspension ! La suspension est de droit !


In [2]:
from transformers import pipeline

# Load the emotion classification pipeline
emotion_classifier = pipeline('text-classification', 
                              model='bhadresh-savani/distilbert-base-uncased-emotion', 
                              tokenizer='bhadresh-savani/distilbert-base-uncased-emotion')

# Classify some sample text
text = "I am very happy today!"
result = emotion_classifier(text)

# Print the predicted emotion and its score
print(result[0]['label'], result[0]['score'])

All model checkpoint layers were used when initializing TFDistilBertForSequenceClassification.

All the layers of TFDistilBertForSequenceClassification were initialized from the model checkpoint at bhadresh-savani/distilbert-base-uncased-emotion.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


joy 0.9986829161643982


In [3]:
import re
def nlp_pipeline(text):

    text = text.lower()
    text = text.replace('\n', ' ').replace('\r', '')
    text = ' '.join(text.split())
    text = re.sub(r"[A-Za-z\.]*[0-9]+[A-Za-z%°\.]*", "", text)
    text = re.sub(r"(\s\-\s|-$)", "", text)
    text = re.sub(r"[,\!\?\%\(\)\/\"]", "", text)
    text = re.sub(r"\&\S*\s", "", text)
    text = re.sub(r"\&", "", text)
    text = re.sub(r"\+", "", text)
    text = re.sub(r"\#", "", text)
    text = re.sub(r"\$", "", text)
    text = re.sub(r"\£", "", text)
    text = re.sub(r"\%", "", text)
    text = re.sub(r"\:", "", text)
    text = re.sub(r"\@", "", text)
    text = re.sub(r"\-", "", text)

    return text
df["text_cleaned"]=df["intervention"].apply(lambda x: nlp_pipeline(x))

In [4]:
small_df = df.sample(100)

In [6]:
emotions = []
cnt = 0
for x in df["text_cleaned"]:
    if len(x.split(" ")) > 100:
        tmp = []
        for i in range(0, len(x.split(" ")), 100):
            tmp.append(emotion_classifier(" ".join(x.split(" ")[i:i+100]))[0])
        emotions.append(tmp)
    else :
        emotions.append(emotion_classifier(x))
    #print(cnt)
    cnt += 1

df["emotions"] = emotions

In [7]:
df.to_csv("tmp/interventions_emotions.csv", index=False)

Test not useful : 

In [41]:
from multiprocessing import Pool

def process_text(text):
    if len(text.split(" ")) > 100:
        tmp = []
        for i in range(0, len(text.split(" ")), 100):
            tmp.append(emotion_classifier(" ".join(text.split(" ")[i:i+100]))[0])
        return tmp
    else :
        return emotion_classifier(text)

def process_dataframe(df):
    with Pool() as pool:
        emotions = pool.map(process_text, df["text_cleaned"])
    df["emotions"] = emotions
    return df
process_dataframe(df)

In [38]:
import swifter

def split_array(arr):
    result = []
    for i in range(0, len(arr), 100):
        result.append(arr[i:i+100])
    return result

small_df["emotions"] = small_df["intervention"].apply(lambda x: 
    emotion_classifier(x) if len(x.split(" ")) < 100 
    else 
    [emotion_classifier(y) for y in split_array(x.split(" "))]
)

End of test

In [2]:
df = pd.read_csv("tmp/interventions_emotions.csv")
df.head()

,intervenant,parti,intervention,text_cleaned,emotions
0,Yaël Braun-Pivet,REN,\r\nL'ordre du jour appelle la discussion du p...,l'ordre du jour appelle la discussion du proje...,"[{'label': 'joy', 'score': 0.635474443435669}]"
1,Danièle Obono,LFI,"\r\nNon, pas question !",non pas question,"[{'label': 'anger', 'score': 0.8457685112953186}]"
2,Louis Boyard,LFI,"\r\nNon, ça ne va pas se passer comme ça ! Vou...",non ça ne va pas se passer comme ça vous n'av...,"[{'label': 'anger', 'score': 0.5996062755584717}]"
3,Yaël Braun-Pivet,REN,"\r\nLa parole est à M. le ministre du travail,...",la parole est à m. le ministre du travail du p...,"[{'label': 'joy', 'score': 0.5436665415763855}]"
4,André Chassaigne,GDR,\r\nSuspension ! La suspension est de droit !,suspension la suspension est de droit,"[{'label': 'anger', 'score': 0.7486376166343689}]"


In [16]:
import ast

df["emotions"] = df["emotions"].apply(lambda x : ast.literal_eval(x))
df.head()

,intervenant,parti,intervention,text_cleaned,emotions
0,Yaël Braun-Pivet,REN,\r\nL'ordre du jour appelle la discussion du p...,l'ordre du jour appelle la discussion du proje...,"[{'label': 'joy', 'score': 0.635474443435669}]"
1,Danièle Obono,LFI,"\r\nNon, pas question !",non pas question,"[{'label': 'anger', 'score': 0.8457685112953186}]"
2,Louis Boyard,LFI,"\r\nNon, ça ne va pas se passer comme ça ! Vou...",non ça ne va pas se passer comme ça vous n'av...,"[{'label': 'anger', 'score': 0.5996062755584717}]"
3,Yaël Braun-Pivet,REN,"\r\nLa parole est à M. le ministre du travail,...",la parole est à m. le ministre du travail du p...,"[{'label': 'joy', 'score': 0.5436665415763855}]"
4,André Chassaigne,GDR,\r\nSuspension ! La suspension est de droit !,suspension la suspension est de droit,"[{'label': 'anger', 'score': 0.7486376166343689}]"


In [21]:
def process_emotions(emotions_list):
    emotions_list = [item for sublist in emotions_list for item in sublist]
    emotions_dict = {}
    for emotion in emotions_list:
        label = emotion['label']
        score = emotion['score']
        if label in emotions_dict:
            emotions_dict[label].append(score)
        else:
            emotions_dict[label] = [score]
    for label in emotions_dict:
        emotions_dict[label] = sum(emotions_dict[label]) / len(emotions_dict[label])
    return emotions_dict

# group by the "parti" column and apply the function to the "emotions" column
result = df.groupby('parti')['emotions'].apply(lambda x: process_emotions(x.tolist()))

# print the result
result

parti            
ECO       anger      0.675104
          joy        0.688860
          sadness    0.560611
          fear       0.480036
          love            NaN
                       ...   
app. SOC  anger      0.702964
          joy        0.635011
          sadness         NaN
          fear            NaN
          love            NaN
Name: emotions, Length: 75, dtype: float64

In [25]:
print(result.to_string(header=None, index=True))

ECO       anger      0.675104
          joy        0.688860
          sadness    0.560611
          fear       0.480036
          love            NaN
GDR       anger      0.656965
          joy        0.663139
          sadness    0.662524
          fear       0.543619
          love            NaN
HOR       anger      0.601994
          joy        0.608279
          sadness    0.990634
          fear            NaN
          love            NaN
LFI       anger      0.668948
          joy        0.662806
          sadness    0.688379
          fear       0.634805
          love            NaN
LIOT      anger      0.660130
          joy        0.628258
          sadness         NaN
          fear       0.480959
          love            NaN
LR        anger      0.642816
          joy        0.687771
          sadness    0.761692
          fear       0.554785
          love            NaN
MODEM     anger      0.671662
          joy        0.680684
          sadness    0.617307
          

In [28]:
df[df["parti"] == "app. LR"]

,intervenant,parti,intervention,text_cleaned,emotions
133,Pierre Cordier,app. LR,\r\nVous disiez le contraire il y a six ans !,vous disiez le contraire il y a six ans,"[{'label': 'joy', 'score': 0.56539386510849}]"
160,Pierre Cordier,app. LR,\r\nC'est la base de la démocratie !,c'est la base de la démocratie,"[{'label': 'anger', 'score': 0.7631848454475403}]"
246,Pierre Cordier,app. LR,\r\nPersonne n'y croit !,personne n'y croit,"[{'label': 'anger', 'score': 0.8199328780174255}]"
253,Pierre Cordier,app. LR,"\r\nIl faudrait mobiliser vos troupes, madame !",il faudrait mobiliser vos troupes madame,"[{'label': 'joy', 'score': 0.5527218580245972}]"
328,Pierre Cordier,app. LR,\r\nC'est vrai !,c'est vrai,"[{'label': 'joy', 'score': 0.44239601492881775}]"
590,Pierre Cordier,app. LR,\r\nTrès bien ! Il a raison !,très bien il a raison,"[{'label': 'joy', 'score': 0.7692698836326599}]"
949,Pierre Cordier,app. LR,\r\nIl a raison !,il a raison,"[{'label': 'joy', 'score': 0.8686319589614868}]"
1096,Pierre Cordier,app. LR,\r\nEh oui !,eh oui,"[{'label': 'anger', 'score': 0.7770354151725769}]"
1111,Pierre Cordier,app. LR,\r\nEh oui !,eh oui,"[{'label': 'anger', 'score': 0.7770354151725769}]"
1382,Pierre Cordier,app. LR,\r\nC'est pareil !,c'est pareil,"[{'label': 'anger', 'score': 0.4531966745853424}]"


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation

text = "Here is an example sentence. Another sentence is this one. This is the last sentence."
stop_words = set(stopwords.words('french'))
stemmer = nltk.stem.PorterStemmer()
sentences = sent_tokenize(text)
words = [word_tokenize(sentence.lower()) for sentence in sentences]
filtered_words = [[stemmer.stem(word) for word in words if word not in stop_words and word not in punctuation] for words in words]

word_freq = {}
for sentence in filtered_words:
    for word in sentence:
        if word in word_freq:
            word_freq[word] += 1
        else:
            word_freq[word] = 1

sentence_scores = {}
for i, sentence in enumerate(filtered_words):
    score = 0
    for word in sentence:
        score += word_freq[word]
    sentence_scores[i] = score

summary_sentences = []
sorted_scores = sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)
for i in range(3):  # Change the number to control the length of the summary
    summary_sentences.append(sentences[sorted_scores[i][0]])
summary = ' '.join(summary_sentences)